In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from scipy.stats import wilcoxon
from sklearn.model_selection import GridSearchCV

In [2]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import LabelSpreading, LabelPropagation

# Set a global random seed
random_state = 42

In [3]:
def data_pre_processing(df):
    timestamp_columns = [col for col in df.columns if "timestamp" in col]
    other_non_imp_column_to_remove = ["id","general_relapse_class"]
    columns_with_all_nan = df.columns[df.isna().all(axis=0)].tolist()
    print("shape of the datafram before dropping columns",df.shape)
    df.drop(timestamp_columns + other_non_imp_column_to_remove + columns_with_all_nan, axis=1, inplace=True)
    print("shape of the datafram after dropping columns",df.shape)
    # # Validate column lists
    categorical = df.select_dtypes(include=['bool', 'object']).columns.tolist()
    numerical = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    # columns_with_all_nan = df.columns[df.isna().all(axis=0)]
    # print("Columns with all NaN",columns_with_all_nan)

    print("Length of categorical and numerical columns:", len(categorical),len(numerical))
    # Apply KNN imputation
    imputer = KNNImputer(n_neighbors=5, weights='uniform')
    imputed_data = imputer.fit_transform(df[numerical])
    print(imputed_data.shape)
    df_imputed_numerical = pd.DataFrame(imputed_data, columns=numerical, index=df.index)
    df[numerical] = df_imputed_numerical
    return pd.get_dummies(df, columns=categorical, drop_first=True)
    

In [4]:
# df = pd.read_csv("/home/mohan/Hospital_TCGA/Data/patient_features.csv")
df = pd.read_csv("/home/mohan/Hospital_TCGA/Data/patient_features_early_stage.csv")
# df.head()
# print(sorted(df.columns.values))
df.rename(columns={'relapse?': 'general_relapse_class'}, inplace=True)
y = df["general_relapse_class"].values
df_encoded = data_pre_processing(df)
# df_encoded.to_csv("sample.csv", sep = ",")
X = df_encoded.values
X.shape
df_encoded.head()

shape of the datafram before dropping columns (1348, 76)
shape of the datafram after dropping columns (1348, 70)
Length of categorical and numerical columns: 59 11
(1348, 11)


,age,chemotherapy@t1_start_time,radiotherapy@t1_duration_days,surgery@t1_time,nb_cig_packs_year,nb_cigs_day,family_lung_cancer,family_other_cancer,diagnosis_ecog,radiotherapy@t1_dose,...,adeno_histology_Micropapillary,adeno_histology_Mucinous,adeno_histology_Others,adeno_histology_Papillary,adeno_histology_Solid,adeno_histology_Unknown,biomarker@diagnosis_type_EGFR,biomarker@diagnosis_type_KRAS,biomarker@diagnosis_type_PDL1,biomarker@diagnosis_type_ROS1
0,68.0,3.0,43.2,2.0,44.6,24.0,0.0,1.0,1.0,58.2,...,0,0,0,0,0,0,0,0,0,0
1,63.0,64.0,34.8,1.0,86.0,40.0,0.0,0.0,2.0,52.6,...,0,0,0,0,0,1,0,0,0,0
2,66.0,28.0,34.6,2.0,60.0,40.0,0.0,0.0,0.0,50.8,...,0,0,0,0,0,0,0,0,0,0
3,81.0,4.8,21.2,2.2,80.0,30.0,0.0,0.0,1.0,41.4,...,0,0,0,0,0,0,0,0,0,0
4,63.0,3.0,38.2,1.0,30.0,20.0,1.0,1.0,0.0,52.2,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Hyperparameter grids
param_grids = {
    "Logistic Regression": {
        "C": [0.01, 0.1, 1],
        "penalty": ["l1", "l2"]
    },
    "Ridge Classifier": {
        "alpha": [0.1, 1, 10, 100]
    },
    "LDA": {
#         "solver": ["svd", "lsqr", "eigen"]
    },
    "QDA": {
        "reg_param": [0.0, 0.1, 0.2, 0.3]
    },
    "SVC": {
#         "C": [0.1, 1, 10],
#         "kernel": ["linear"],
#         "degree": [2, 3, 4]  # Only used if kernel is 'poly'
    },
    "KNeighbors": {
        "n_neighbors": [3, 5, 7, 9],
        "metric": ["euclidean"]
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [10, 20, 30]
    },
    "Gradient Boosting": {
        "n_estimators": [100, 200, 300],
        "learning_rate": [0.01, 0.1],
        
    },
    "Gaussian NB": {},
    "Decision Tree": {
        "max_depth": [5, 10, 20],
        "min_samples_split": [2, 5, 10]
    },
    "MLP Classifier": {
        "hidden_layer_sizes": [(50,), (100,), (50, 50)],
        "activation": ["relu"]
    },
    "Label Spreading": {
        "alpha": [0.01, 0.1, 0.2, 0.5]
    },
    "Label Propagation": {
        
    },
    "AdaBoost": {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.01, 0.1, 1]
    }
}

In [6]:
# Define the classifiers
classifiers = {
    "Logistic Regression": LogisticRegression(max_iter=10000,n_jobs=-1, random_state= random_state),
    "Ridge Classifier": RidgeClassifier(),
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "SVC": SVC(probability=True,random_state=random_state),
    "KNeighbors": KNeighborsClassifier(n_jobs=-1),
    "Random Forest": RandomForestClassifier(random_state=random_state,n_jobs=-1),
    "Gradient Boosting": GradientBoostingClassifier(random_state=random_state),
    "Gaussian NB": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(random_state=random_state),
    "MLP Classifier": MLPClassifier(max_iter=1000,random_state=random_state),
    "Label Spreading": LabelSpreading(kernel='knn', max_iter=1000,n_jobs=-1),  # Specify kernel for LabelSpreading
    "Label Propagation": LabelPropagation(kernel='knn', max_iter=1000,n_jobs=-1),  # Specify kernel for LabelPropagation
    "AdaBoost": AdaBoostClassifier(algorithm="SAMME", random_state=random_state)
   
}

# Initialize 5-Fold Stratified CV
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
# cv = KFold(n_splits=10,random_state=42,shuffle=True)
# Initialize dict to store scores
scores = {name: {"AUC-PR": [], "AUC-ROC": [], "Precision": [], "Recall": [], "F1-score": [],"Accuracy": []} for name in classifiers}

# Perform 5-Fold CV and calculate metrics
for name, classifier in classifiers.items():
    print(f"Evaluating {name}")
    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        grid_search = GridSearchCV(classifier, param_grids[name], cv=2, scoring='roc_auc')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred =  best_model.predict(X_test)
#         classifier.fit(X_train, y_train)
#         y_pred = classifier.predict(X_test)

        # Check if classifier supports `predict_proba`
        if hasattr(classifier, "predict_proba"):
            y_pred_proba = best_model.predict_proba(X_test)[:, 1]
            scores[name]["AUC-PR"].append(average_precision_score(y_test, y_pred_proba))
            scores[name]["AUC-ROC"].append(roc_auc_score(y_test, y_pred_proba))
        else:
            # If no `predict_proba`, mark AUC scores as not applicable
            scores[name]["AUC-PR"].append(np.nan)
            scores[name]["AUC-ROC"].append(np.nan)

        # Calculate Precision, Recall, and F1-score
        scores[name]["Precision"].append(precision_score(y_test, y_pred, zero_division=0))
        scores[name]["Recall"].append(recall_score(y_test, y_pred, zero_division=0))
        scores[name]["F1-score"].append(f1_score(y_test, y_pred, zero_division=0))
        scores[name]["Accuracy"].append(accuracy_score(y_test, y_pred))

# Convert scores to a pandas DataFrame for nicer display, formatting mean ± std dev
# Assuming `scores` dictionary is already populated
# Initialize a list to store raw scores for sorting
raw_results = []

for name in scores:
    for metric, values in scores[name].items():
        if metric == "AUC-PR":
            metric_scores = [score for score in values if not np.isnan(score)]
            if metric_scores:
                avg_score = np.mean(metric_scores)
                # Store raw mean scores for AUC-PR for sorting
                raw_results.append((name, avg_score))

# Sort classifiers based on mean AUC-PR score in descending order
sorted_classifiers = sorted(raw_results, key=lambda x: x[1], reverse=False)

# Now, format and display results in sorted order
results = []

for name, _ in sorted_classifiers:
    result_entry = {"Classifier": name}
    for metric in scores[name]:
        metric_scores = [score for score in scores[name][metric] if not np.isnan(score)]
        if metric_scores:
            avg_score = np.mean(metric_scores)
            std_dev = np.std(metric_scores)
            result_entry[metric] = f"{avg_score:.3f} ± {std_dev:.3f}"
        else:
            result_entry[metric] = "N/A"
    results.append(result_entry)

# Create DataFrame from sorted and formatted results
results_df = pd.DataFrame(results)
results_df.set_index("Classifier", inplace=True)


Evaluating Logistic Regression


/home/mohan/anaconda3/envs/textemb/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
6 fits failed out of a total of 12.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/home/mohan/anaconda3/envs/textemb/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/mohan/anaconda3/envs/textemb/lib/python3.8/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/mohan/anaconda3/envs/textemb/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line

Evaluating Ridge Classifier
Evaluating LDA
Evaluating QDA


/home/mohan/anaconda3/envs/textemb/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/mohan/anaconda3/envs/textemb/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/mohan/anaconda3/envs/textemb/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/mohan/anaconda3/envs/textemb/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/mohan/anaconda3/envs/textemb/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/mohan/anaconda3/envs/textemb/lib/python3.8/site-packages/sklearn/discriminant_analysis.py

Evaluating SVC
Evaluating KNeighbors
Evaluating Random Forest
Evaluating Gradient Boosting
Evaluating Gaussian NB
Evaluating Decision Tree
Evaluating MLP Classifier
Evaluating Label Spreading
Evaluating Label Propagation
Evaluating AdaBoost


In [7]:
results_df

,AUC-PR,AUC-ROC,Precision,Recall,F1-score,Accuracy
Classifier,,,,,,
Label Propagation,0.458 ± 0.048,0.579 ± 0.048,0.492 ± 0.090,0.271 ± 0.071,0.348 ± 0.081,0.634 ± 0.037
KNeighbors,0.458 ± 0.046,0.578 ± 0.051,0.502 ± 0.078,0.255 ± 0.063,0.336 ± 0.070,0.638 ± 0.028
Label Spreading,0.478 ± 0.056,0.581 ± 0.046,0.516 ± 0.109,0.249 ± 0.068,0.332 ± 0.077,0.640 ± 0.037
SVC,0.497 ± 0.070,0.607 ± 0.051,0.000 ± 0.000,0.000 ± 0.000,0.000 ± 0.000,0.636 ± 0.002
Gaussian NB,0.514 ± 0.052,0.668 ± 0.038,0.507 ± 0.043,0.635 ± 0.102,0.559 ± 0.048,0.638 ± 0.046
Decision Tree,0.635 ± 0.075,0.719 ± 0.058,0.790 ± 0.088,0.387 ± 0.107,0.507 ± 0.109,0.736 ± 0.035
MLP Classifier,0.640 ± 0.088,0.739 ± 0.065,0.619 ± 0.114,0.495 ± 0.173,0.522 ± 0.117,0.691 ± 0.045
QDA,0.666 ± 0.075,0.758 ± 0.063,0.725 ± 0.070,0.399 ± 0.063,0.512 ± 0.058,0.725 ± 0.027
LDA,0.690 ± 0.057,0.770 ± 0.040,0.648 ± 0.065,0.521 ± 0.083,0.576 ± 0.073,0.723 ± 0.039


In [8]:

mean_auc_pr_scores = {classifier: np.nanmean(scores[classifier]["AUC-PR"]) for classifier in classifiers}
best_model = max(mean_auc_pr_scores, key=mean_auc_pr_scores.get)
print(f"The best performing model based on AUC-PR is: {best_model}")

# Perform Wilcoxon signed-rank tests between the best model and all others
p_values = pd.DataFrame(index=[best_model], columns=classifiers.keys())
auc_pr_scores = {classifier: scores[classifier]["AUC-PR"] for classifier in classifiers}

for classifier in classifiers.keys():
    if classifier != best_model:
        stat, p = wilcoxon(auc_pr_scores[best_model], auc_pr_scores[classifier])
        p_values.loc[best_model, classifier] = p
    else:
        p_values.loc[best_model, classifier] = np.nan  # Self-comparison

# Formatting the DataFrame to highlight p-values <= 0.05
def highlight_p_values(val):
    color = 'bold' if val <= 0.05 else ''
    return f'{color} {val:.3f}' if not pd.isna(val) else ''

formatted_p_values = p_values.astype(float).applymap(highlight_p_values)

print("P-values from Wilcoxon signed-rank tests comparing the best model against others:")
print(formatted_p_values)

The best performing model based on AUC-PR is: Random Forest
P-values from Wilcoxon signed-rank tests comparing the best model against others:
              Logistic Regression Ridge Classifier     LDA         QDA  \
Random Forest               0.322                    0.160  bold 0.027   

                      SVC  KNeighbors Random Forest Gradient Boosting  \
Random Forest  bold 0.002  bold 0.002                           0.064   

              Gaussian NB Decision Tree MLP Classifier Label Spreading  \
Random Forest  bold 0.002    bold 0.002     bold 0.020      bold 0.002   

              Label Propagation AdaBoost  
Random Forest        bold 0.002    0.084  


/tmp/ipykernel_35360/2778090644.py:1: RuntimeWarning: Mean of empty slice
  mean_auc_pr_scores = {classifier: np.nanmean(scores[classifier]["AUC-PR"]) for classifier in classifiers}


In [9]:
formatted_p_values

,Logistic Regression,Ridge Classifier,LDA,QDA,SVC,KNeighbors,Random Forest,Gradient Boosting,Gaussian NB,Decision Tree,MLP Classifier,Label Spreading,Label Propagation,AdaBoost
Random Forest,0.322,,0.160,bold 0.027,bold 0.002,bold 0.002,,0.064,bold 0.002,bold 0.002,bold 0.020,bold 0.002,bold 0.002,0.084


In [10]:
temp_df = results_df 
temp_df = temp_df.drop(["Precision", "Recall", "F1-score"], axis = 1)
temp_df

,AUC-PR,AUC-ROC,Accuracy
Classifier,,,
Label Propagation,0.458 ± 0.048,0.579 ± 0.048,0.634 ± 0.037
KNeighbors,0.458 ± 0.046,0.578 ± 0.051,0.638 ± 0.028
Label Spreading,0.478 ± 0.056,0.581 ± 0.046,0.640 ± 0.037
SVC,0.497 ± 0.070,0.607 ± 0.051,0.636 ± 0.002
Gaussian NB,0.514 ± 0.052,0.668 ± 0.038,0.638 ± 0.046
Decision Tree,0.635 ± 0.075,0.719 ± 0.058,0.736 ± 0.035
MLP Classifier,0.640 ± 0.088,0.739 ± 0.065,0.691 ± 0.045
QDA,0.666 ± 0.075,0.758 ± 0.063,0.725 ± 0.027
LDA,0.690 ± 0.057,0.770 ± 0.040,0.723 ± 0.039
